# Import Packages

In [3]:
import duckdb
import plotly.express as px
import pandas as pd

# Connect to Local Database

In [87]:
# Create connection to local duckdb database
con = duckdb.connect('../db/duck.db')

IOException: IO Error: Could not set lock on file "/Users/kul/Dev/featlib/notebooks/../db/duck.db": Conflicting lock is held in /Users/kul/.pyenv/versions/3.12.8/bin/python3.12 (PID 60089) by user kul. See also https://duckdb.org/docs/connect/concurrency

# View Database

In [6]:
con.sql("SHOW DATABASES;")

┌───────────────┐
│ database_name │
│    varchar    │
├───────────────┤
│ duck          │
└───────────────┘

In [80]:
con.sql("SHOW TABLES;")

┌───────────────────────────────────┐
│               name                │
│              varchar              │
├───────────────────────────────────┤
│ balance_sheet_assets              │
│ balance_sheet_equity              │
│ balance_sheet_liabilities         │
│ cash_flow_statement               │
│ dividend                          │
│ earnings_calendar                 │
│ eps_estimate                      │
│ eps_history                       │
│ features                          │
│ features_temp                     │
│       ·                           │
│       ·                           │
│       ·                           │
│ sales_estimate                    │
│ split                             │
│ split_adj                         │
│ symbol                            │
│ tmp_pricing01_inc_splits          │
│ tmp_pricing02_remsym_misshist     │
│ tmp_pricing03_remsym_zeros        │
│ tmp_pricing04_resampled           │
│ tmp_pricing05_rem_sym_insuff_hist │
│ wolfe     

# Create Split Adjustment Factor

In [8]:
con.sql("""
CREATE OR REPLACE TABLE split_adj AS
    -- Get the unique symbols from the split table
    WITH base_table AS (
        SELECT DISTINCT act_symbol FROM split
    ),
    -- Create a new table with a row for each symbol and a NULL split for date 2000-01-01
    new_split_table AS (
        SELECT 
            act_symbol,
            DATE '2000-01-01' as ex_date,
            NULL as to_factor,
            NULL as for_factor,
        FROM base_table
        UNION ALL
        SELECT * FROM split
    ),
    -- Calculate the adjustment factor
    split_factors AS (
        -- Calculate adjustment factors and order the rows
        SELECT
            act_symbol,
            ex_date AS adj_date,
            for_factor / to_factor AS adj_factor
        FROM new_split_table
    ),
    -- Order the factors and assign end dates
    ordered_factors AS (
        -- Assign end dates and ensure chronological order
        SELECT
            act_symbol,
            adj_date,
            CAST (LEAD(adj_date) OVER (PARTITION BY act_symbol ORDER BY adj_date ASC) - INTERVAL '1 DAY' AS DATE) AS end_adj_date,
            adj_factor,
            ROW_NUMBER() OVER (PARTITION BY act_symbol ORDER BY adj_date DESC) AS reverse_row_num
        FROM split_factors
    ),
    -- Calculate the cumulative adjustment factor
    cumulative_factors AS (
        -- Compute cumulative adjustment factors in reverse order
        SELECT
            act_symbol,
            adj_date,
            end_adj_date,
            adj_factor,
            -- Reverse chronological cumulative multiplication
            CASE 
                WHEN end_adj_date IS NULL 
                    THEN 1
                WHEN lead(end_adj_date) OVER (PARTITION BY act_symbol ORDER BY adj_date ASC) IS NULL
                    THEN lead(adj_factor) OVER (PARTITION BY act_symbol ORDER BY adj_date ASC)
                ELSE product(adj_factor) OVER (PARTITION BY act_symbol ORDER BY reverse_row_num ASC ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING)
            END AS cum_adj_factor
        FROM ordered_factors
    )
    -- Final output
    SELECT
        act_symbol,
        adj_date,
        end_adj_date,
        adj_factor,
        cum_adj_factor
    FROM cumulative_factors
    ORDER BY adj_date ASC;
""")

# Pricing

### Create Pricing inc Splits Table

In [9]:
con.sql("""
-- 16473
CREATE OR REPLACE TEMP TABLE tmp_pricing01_inc_splits AS (
    with ohlcv_split as (
        select 
            a.date,
            a.act_symbol,
            a.open * COALESCE(f.cum_adj_factor, 1) as open,
            a.high * COALESCE(f.cum_adj_factor, 1) as high,
            a.low * COALESCE(f.cum_adj_factor, 1) as low,
            a.close * COALESCE(f.cum_adj_factor, 1) as close,
            a.volume / COALESCE(f.cum_adj_factor, 1) as volume,
        FROM ohlcv a
        LEFT JOIN split_adj f on a.act_symbol=f.act_symbol AND a.date >= f.adj_date AND (a.date <= f.end_adj_date OR f.end_adj_date IS NULL)
    )
    select * from ohlcv_split  
);
"""
)

### Remove Symbols With History Gaps

In [31]:
con.sql(
    """
    -- 13,836
    -- Remove symbols with gaps > 14 days
    CREATE OR REPLACE TEMP TABLE tmp_pricing02_remsym_misshist AS (
        WITH day_difference AS (
        SELECT
            *,
            LEAD(date) OVER (PARTITION BY act_symbol ORDER BY date) - date AS day_count
        FROM
            tmp_pricing01_inc_splits
        ),
        filtered_symbols AS (
            SELECT
                act_symbol
            FROM
                day_difference
            WHERE
                day_count > 14
            GROUP BY
                act_symbol
        )
        SELECT
            * exclude (day_count)
        FROM
            day_difference
        WHERE
            act_symbol NOT IN (SELECT act_symbol FROM filtered_symbols)
        AND open IS NOT NULL AND high IS NOT NULL AND low IS NOT NULL AND close IS NOT NULL AND volume IS NOT NULL
    );
    """
)

### Remove Zero Prices

In [32]:
con.sql(
    """
    -- Remove symbols which have more than 5 zero values for ohlcv
    -- Massage the data to replace zeros with the avg/min/max of the previous 5 days depending on the column
    CREATE OR REPLACE TEMP TABLE tmp_pricing03_remsym_zeros AS (
        WITH zero_close_count AS (
        SELECT
            act_symbol,
            COUNT(*) AS zero_close_count
        FROM
            tmp_pricing02_remsym_misshist
        WHERE
            close = 0
        GROUP BY
            act_symbol
        HAVING
            zero_close_count > 5
        ),
        zero_low_count AS (
            SELECT
                act_symbol,
                COUNT(*) AS zero_low_count
            FROM
                tmp_pricing02_remsym_misshist
            WHERE
                low = 0
            GROUP BY
                act_symbol
            HAVING
                zero_low_count > 5
        ),
        zero_open_count AS (
            SELECT
                act_symbol,
                COUNT(*) AS zero_open_count
            FROM
                tmp_pricing02_remsym_misshist
            WHERE
                open = 0
            GROUP BY
                act_symbol
            HAVING
                zero_open_count > 5
        ),
        zero_high_count AS (
            SELECT
                act_symbol,
                COUNT(*) AS zero_high_count
            FROM
                tmp_pricing02_remsym_misshist
            WHERE
                high = 0
            GROUP BY
                act_symbol
            HAVING
                zero_high_count > 5
        ),
        zero_volume_count AS (
            SELECT
                act_symbol,
                COUNT(*) AS zero_volume_count
            FROM
                tmp_pricing02_remsym_misshist
            WHERE
                volume = 0
            GROUP BY
                act_symbol
            HAVING
                zero_volume_count > 5
        ),
        filtered_symbols AS (
            SELECT act_symbol FROM zero_close_count
            UNION
            SELECT act_symbol FROM zero_low_count
            UNION
            SELECT act_symbol FROM zero_open_count
            UNION
            SELECT act_symbol FROM zero_high_count
            UNION
            SELECT act_symbol FROM zero_volume_count
        ),
        remove_symbols_and_fix_zeros AS (
            SELECT
                date,
                act_symbol,
                COALESCE(
                    NULLIF(open, 0),
                    AVG(open) OVER (
                        PARTITION BY act_symbol
                        ORDER BY date asc
                        ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING
                    )
                ) AS open,
                COALESCE(
                    NULLIF(high, 0),
                    MAX(high) OVER (
                        PARTITION BY act_symbol
                        ORDER BY date asc
                        ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING
                    )
                ) AS high,
                COALESCE(
                    NULLIF(low, 0),
                    AVG(NULLIF(low, 0)) OVER (
                        PARTITION BY act_symbol
                        ORDER BY date asc
                        ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING
                    )
                ) AS low,
                COALESCE(
                    NULLIF(close, 0),
                    AVG(close) OVER (
                        PARTITION BY act_symbol
                        ORDER BY date asc
                        ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING
                    )
                ) AS close,
                COALESCE(
                    NULLIF(volume, 0),
                    AVG(NULLIF(volume, 0)) OVER (
                        PARTITION BY act_symbol
                        ORDER BY date asc
                        ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING
                    )
                ) AS volume
            ------------ OLD ------------
            --SELECT
            --    date,
            --    act_symbol,
            --    COALESCE(NULLIF(open, 0), LAG(open) OVER (PARTITION BY act_symbol ORDER BY date)) AS open,
            --    COALESCE(NULLIF(high, 0), LAG(high) OVER (PARTITION BY act_symbol ORDER BY date)) AS high,
            --    COALESCE(NULLIF(low, 0), LAG(low) OVER (PARTITION BY act_symbol ORDER BY date)) AS low,
            --    COALESCE(NULLIF(close, 0), LAG(close) OVER (PARTITION BY act_symbol ORDER BY date)) AS close,
            --    COALESCE(NULLIF(volume, 0), LAG(volume) OVER (PARTITION BY act_symbol ORDER BY date)) AS volume
            ------------ OLD ------------
            FROM
                tmp_pricing02_remsym_misshist
            WHERE
                act_symbol NOT IN (SELECT act_symbol FROM filtered_symbols)
        )
        SELECT
            *
        FROM remove_symbols_and_fix_zeros
        where high >= low
    );
    """
)

### Resample to Business Day Frequency

In [60]:
con.sql(
"""
CREATE OR REPLACE TEMP TABLE tmp_pricing04_resampled AS (
    -- Step 1: Identify the max date for each symbol
    WITH max_dates AS (
        SELECT
            act_symbol,
            MAX(date) AS max_date
        FROM
            tmp_pricing03_remsym_zeros
        GROUP BY
            act_symbol
    ),

    -- Step 2: Generate a business day calendar for the date range in your data
    date_series AS (
    SELECT
        UNNEST(GENERATE_SERIES(
            MIN(date), -- Start date of your data
            MAX(date), -- End date of your data
            INTERVAL '1' DAY
        )) AS date
    FROM
        tmp_pricing03_remsym_zeros
    ),
    business_days AS (
        SELECT
            date
        FROM
            date_series
        WHERE
            EXTRACT(DOW FROM date) NOT IN (0, 6) -- Exclude Sundays (0) and Saturdays (6)
        ORDER BY
            date
    ),

    -- Step 3: Create a complete business day calendar for each symbol up to its max date
    symbol_calendar AS (
        SELECT
            bd.date,
            md.act_symbol
        FROM
            business_days bd
        CROSS JOIN
            max_dates md
        WHERE
            bd.date <= md.max_date
    ),

    -- Step 4: Join the calendar with your data and forward-fill missing values
    resampled_data AS (
        SELECT
            sc.date,
            sc.act_symbol,
            COALESCE(p.open, LAST_VALUE(p.open IGNORE NULLS) OVER (PARTITION BY sc.act_symbol ORDER BY sc.date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)) AS open,
            COALESCE(p.high, LAST_VALUE(p.high IGNORE NULLS) OVER (PARTITION BY sc.act_symbol ORDER BY sc.date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)) AS high,
            COALESCE(p.low, LAST_VALUE(p.low IGNORE NULLS) OVER (PARTITION BY sc.act_symbol ORDER BY sc.date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)) AS low,
            COALESCE(p.close, LAST_VALUE(p.close IGNORE NULLS) OVER (PARTITION BY sc.act_symbol ORDER BY sc.date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)) AS close,
            COALESCE(p.volume, LAST_VALUE(p.volume IGNORE NULLS) OVER (PARTITION BY sc.act_symbol ORDER BY sc.date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)) AS volume
        FROM
            symbol_calendar sc
        LEFT JOIN
            tmp_pricing03_remsym_zeros p
        ON
            sc.date = p.date AND sc.act_symbol = p.act_symbol
    )

    -- Final Output
    SELECT * FROM resampled_data
    where open is not null and high is not null and low is not null and close is not null and volume is not null
    and isnan(open) = false and isnan(high) = false and isnan(low) = false and isnan(close) = false and isnan(volume) = false
    and isinf(open) = false and isinf(high) = false and isinf(low) = false and isinf(close) = false and isinf(volume) = false
    ORDER BY act_symbol, date
);
"""
)

### Remove Symbols with Insufficient History (3 months / 63 trading days)

In [58]:
con.sql("""
CREATE OR REPLACE TEMP TABLE tmp_pricing05_rem_sym_insuff_hist AS (
    WITH symbol_counts AS (
        SELECT 
            act_symbol, 
            COUNT(*) AS num_days 
        FROM tmp_pricing04_resampled
        GROUP BY act_symbol
    )
    SELECT * FROM tmp_pricing04_resampled
    WHERE act_symbol NOT IN (
        SELECT act_symbol 
        FROM symbol_counts 
        WHERE num_days < 63
    )
);
"""
)

### Create Pricing Table

In [65]:
con.sql("""
CREATE OR REPLACE TABLE pricing AS (
    SELECT * FROM tmp_pricing05_rem_sym_insuff_hist
);
"""
)

# Data Quality Check

In [66]:
con.sql("with cte as (select distinct act_symbol from pricing) select count(*) from cte;")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        11547 │
└──────────────┘

In [160]:
con.sql("select * from ohlcv where date = '2025-01-11' limit 10;")

┌──────┬────────────┬──────────────┬──────────────┬──────────────┬──────────────┬────────┐
│ date │ act_symbol │     open     │     high     │     low      │    close     │ volume │
│ date │  varchar   │ decimal(7,2) │ decimal(7,2) │ decimal(7,2) │ decimal(7,2) │ int64  │
├──────┴────────────┴──────────────┴──────────────┴──────────────┴──────────────┴────────┤
│                                         0 rows                                         │
└────────────────────────────────────────────────────────────────────────────────────────┘

In [81]:
con.sql("""
select 
    *
from tmp_pricing04_resampled a
where a.high < a.low or a.volume = 0 or a.open = 0 or a.high = 0 or a.low = 0 or a.close = 0;
""")

┌───────────┬────────────┬────────┬────────┬────────┬────────┬────────┐
│   date    │ act_symbol │  open  │  high  │  low   │ close  │ volume │
│ timestamp │  varchar   │ double │ double │ double │ double │ double │
├───────────┴────────────┴────────┴────────┴────────┴────────┴────────┤
│                               0 rows                                │
└─────────────────────────────────────────────────────────────────────┘

In [83]:
con.sql("""
SELECT * FROM pricing
    where open is null  or high is null or low is null or close is null or volume is null
""")

┌───────────┬────────────┬────────┬────────┬────────┬────────┬────────┐
│   date    │ act_symbol │  open  │  high  │  low   │ close  │ volume │
│ timestamp │  varchar   │ double │ double │ double │ double │ double │
├───────────┴────────────┴────────┴────────┴────────┴────────┴────────┤
│                               0 rows                                │
└─────────────────────────────────────────────────────────────────────┘

In [70]:
con.sql("""select * from pricing 
where isnan(open) = true or isnan(high) = true or isnan(low) = true or isnan(close) = true or isnan(volume) = true;""")

┌───────────┬────────────┬────────┬────────┬────────┬────────┬────────┐
│   date    │ act_symbol │  open  │  high  │  low   │ close  │ volume │
│ timestamp │  varchar   │ double │ double │ double │ double │ double │
├───────────┴────────────┴────────┴────────┴────────┴────────┴────────┤
│                               0 rows                                │
└─────────────────────────────────────────────────────────────────────┘

In [71]:
con.sql("""select * from pricing 
where isinf(open) = true or isinf(high) = true or isinf(low) = true or isinf(close) = true or isinf(volume) = true;""")

┌───────────┬────────────┬────────┬────────┬────────┬────────┬────────┐
│   date    │ act_symbol │  open  │  high  │  low   │ close  │ volume │
│ timestamp │  varchar   │ double │ double │ double │ double │ double │
├───────────┴────────────┴────────┴────────┴────────┴────────┴────────┤
│                               0 rows                                │
└─────────────────────────────────────────────────────────────────────┘

## Test Plot

In [85]:
# Step 1: Query the DuckDB table into a Pandas DataFrame
query = """
SELECT 
    *,
    log(close) as log_close
from pricing a 
where a.act_symbol ='NVDA'
order by date asc
"""
df = con.execute(query).fetchdf()
df.tail()

# Step 2: Plot the data using Plotly Express
fig = px.line(df, x='date', y='close')
fig.show()

# Close Database Connection

In [86]:
con.close()